In [1]:
import numpy as np
import pandas as pd

In [2]:
geo_taxi = pd.read_csv('taxi_tickets_geocoded.csv')
geo_taxi = geo_taxi[['Ticket Number', 
                     'License Plate Number', 
                     'License Plate State',
                     'License Plate Type',
                     'Ticket Make',
                     'Issue Date',
                     'Violation Location',
                     'Violation Description',
                     'Badge',
                     'Unit',
                     'Ticket Queue',
                     'Hearing Dispo', 
                     'Violation Address',
                     'lat',
                     'lon'
                    ]]
geo_taxi = geo_taxi.rename(columns={
        'Ticket Number': 'ticket_num',
        'License Plate Number': 'plate_num',
        'License Plate State': 'plate_state',
        'License Plate Type': 'plate_type',
        'Ticket Make': 'car_make',
        'Issue Date': 'issue_date',
        'Violation Location': 'location',
        'Violation Description': 'description',
        'Badge': 'badge',
        'Unit': 'unit',
        'Ticket Queue': 'ticket_queue',
        'Hearing Dispo': 'hearing_disp',
        'Violation Address': 'address',
    })
geo_taxi.to_csv('postgis_taxi_tickets.csv', index=False)

/usr/local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
geocoded_taxi = pd.read_csv('taxi_tickets_to_geocode_output.csv')
print(geocoded_taxi.dtypes)
print(geocoded_taxi.shape)
print(geocoded_taxi['lon'].isnull().sum())
geocoded_taxi.head()

Ticket Number              int64
License Plate Number      object
License Plate State       object
License Plate Type        object
Ticket Make               object
Issue Date                object
Violation Location        object
Violation Code            object
Violation Description     object
Badge                     object
Unit                      object
Ticket Queue              object
Hearing Dispo             object
Violation Address         object
lat                      float64
lon                      float64
dtype: object
(261936, 16)
0


/Users/pjsier/.virtualenvs/dna3/lib/python3.4/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Ticket Number,License Plate Number,License Plate State,License Plate Type,Ticket Make,Issue Date,Violation Location,Violation Code,Violation Description,Badge,Unit,Ticket Queue,Hearing Dispo,Violation Address,lat,lon
0,55884862,4079TX,IL,TXI,FORD,01/02/2009 10:05 am,5210 N BROADWAY,0964190,EXPIRED METER OR OVERSTAY,8117,20,Paid,NaN,"5210 N BROADWAY CHICAGO, IL",41.976507,-87.659944
1,54574383,5141TX,IL,TXI,MERC,01/06/2009 06:53 am,4707 N PULASKI,0964170B,"TRUCK,MOTOR HOME, BUS BUSINESS STREET",12769,17,Paid,NaN,"4707 N PULASKI CHICAGO, IL",41.966575,-87.727904
2,55388795,25495TX,IL,TXI,MERC,01/05/2009 04:19 am,5721 W EASTWOOD,0964170A,"TRUCK,RV,BUS, OR TAXI RESIDENTIAL STREET",13250,16,Notice,NaN,"5721 W EASTWOOD CHICAGO, IL",41.964930,-87.770331
3,9177658970,25368TX,IL,TXI,OLDS,01/10/2009 10:30 am,6410 N CLAREMONT AV,0964190,EXPIRED METER OR OVERSTAY,197,498,Paid,NaN,"6410 N CLAREMONT AV CHICAGO, IL",41.997942,-87.688741
4,55835485,3001TX,IL,TXI,FORD,01/01/2009 06:35 pm,36 W 26TH,0976160F,EXPIRED PLATES OR TEMPORARY REGISTRATION,19503,21,Bankruptcy,NaN,"36 W 26TH CHICAGO, IL",41.845162,-87.628529


In [5]:
failed_geo = pd.read_csv('failed_geocoding.csv')
print(failed_geo.shape)
failed_geo.head()

(18, 18)


,X,Y,Ticket Num,License Pl,License _1,License _2,Ticket Mak,Issue Date,Violation,Violatio_1,Violatio_2,Badge,Unit,Ticket Que,Hearing Di,Violatio_3,lat,lon
0,-88.901443,38.087223,59392077,1916TX,IL,TXI,FORD,02/27/2011 10:00 pm,6354 W HUTINGTON,0964170A,"TRUCK,RV,BUS, OR TAXI RESIDENTIAL STREET",7355,16,Paid,Liable,"6354 W HUTINGTON CHICAGO, IL",38.087223,-88.901443
1,-88.717168,40.449980,63060474,982TX,IL,TXI,SCIO,05/18/2013 03:03 pm,5358 S ELLSWORTH DR,0964140B,PARK OR STAND IN BUS/TAXI/CARRIAGE STAND,14988,2,Dismissed,Not Liable,"5358 S ELLSWORTH DR CHICAGO, IL",40.449980,-88.717168
2,-88.717168,40.449980,63547329,H258550,IL,TXI,CHRY,09/22/2013 04:49 pm,5415 S ELLSWORTH,0964110E,PARK OR STAND ON PARKWAY,5920,2,Define,NaN,"5415 S ELLSWORTH CHICAGO, IL",40.449980,-88.717168
3,-88.717168,40.449980,63066120,5442914,IL,TXI,CHEV,06/09/2013 03:42 pm,5446 S ELLSWORTH DR,0912060,"STAND, PARK, OR OTHER USE OF BUS LANE",3005,2,Paid,NaN,"5446 S ELLSWORTH DR CHICAGO, IL",40.449980,-88.717168
4,-88.717168,40.449980,63484925,2400582,IL,TXI,FORD,08/31/2013 03:43 pm,5159 S ELLSWORTH DR,0964100C,BLOCK ACCESS/ALLEY/DRIVEWAY/FIRELANE,19968,2,Dismissed,DOR Withdraw,"5159 S ELLSWORTH DR CHICAGO, IL",40.449980,-88.717168


In [8]:
failed_geo.apply(lambda x: print(x['Violatio_3']), axis=1)

6354 W HUTINGTON CHICAGO, IL
5358 S ELLSWORTH DR CHICAGO, IL
5415 S ELLSWORTH CHICAGO, IL
5446 S ELLSWORTH DR CHICAGO, IL
5159 S ELLSWORTH DR CHICAGO, IL
561  LINCOLN CHICAGO, IL
2022 W GRANDVILLE CHICAGO, IL
5329 N -LINCOLN CHICAGO, IL
5535  LINCOLN CHICAGO, IL
5340  LINCOLN CHICAGO, IL
5118 S ELLSWORTH CHICAGO, IL
5323  LINCOLN CHICAGO, IL
5630  LINCOLN CHICAGO, IL
2249 W LINCOLN CHICAGO, IL
458 S SPRINGIFELD CHICAGO, IL
5539  LINCOLN CHICAGO, IL
6448 S CHAMPAIN CHICAGO, IL
1132 W GRANDVILLE CHICAGO, IL


0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
dtype: object

In [9]:
fixed_geo = pd.read_csv('fixed_geo.csv')
print(fixed_geo.dtypes)
fixed_geo

original address     object
returned address     object
latitude            float64
longitude           float64
accuracy              int64
dtype: object


,original address,returned address,latitude,longitude,accuracy
0,"6354 W HUTINGTON CHICAGO, IL","6354 W Huntington St, Chicago, IL 60646, USA",41.992800,-87.786801,9
1,"5358 S ELLSWORTH DR CHICAGO, IL","5358 S Ellsworth Dr, Chicago, IL 60615, USA",41.798336,-87.613949,8
2,"5415 S ELLSWORTH CHICAGO, IL","5415 S Ellsworth Dr, Chicago, IL 60615, USA",41.798336,-87.613949,8
3,"5446 S ELLSWORTH DR CHICAGO, IL","5446 S Ellsworth Dr, Chicago, IL 60615, USA",41.798336,-87.613949,8
4,"5159 S ELLSWORTH DR CHICAGO, IL","5159 S Ellsworth Dr, Chicago, IL 60615, USA",41.798336,-87.613949,8
5,"561 LINCOLN CHICAGO, IL","561 Lincoln Ave, West Chicago, IL 60185, USA",41.876291,-88.216590,8
6,"2022 W GRANDVILLE CHICAGO, IL","2022 W Granville Ave, Chicago, IL 60660, USA",41.994272,-87.680188,8
7,"5329 N -LINCOLN CHICAGO, IL","5329 N Lincoln Ave, Chicago, IL 60625, USA",41.978535,-87.692142,9
8,"5535 LINCOLN CHICAGO, IL","5535 N Lincoln Ave, Chicago, IL 60625, USA",41.982474,-87.693531,9
9,"5340 LINCOLN CHICAGO, IL","5340 N Lincoln Ave, Chicago, IL 60625, USA",41.978864,-87.692825,9


In [10]:
geocoded_taxi[geocoded_taxi['lat'] < 41].count()

Ticket Number            10
License Plate Number     10
License Plate State      10
License Plate Type       10
Ticket Make              10
Issue Date               10
Violation Location       10
Violation Code           10
Violation Description    10
Badge                    10
Unit                     10
Ticket Queue             10
Hearing Dispo             3
Violation Address        10
lat                      10
lon                      10
dtype: int64

In [13]:
geocoded_taxi[geocoded_taxi['lon'] < -88].count()

Ticket Number            22
License Plate Number     22
License Plate State      22
License Plate Type       22
Ticket Make              22
Issue Date               22
Violation Location       22
Violation Code           22
Violation Description    22
Badge                    22
Unit                     22
Ticket Queue             22
Hearing Dispo             6
Violation Address        22
lat                      22
lon                      22
dtype: int64

In [14]:
geo_fix = fixed_geo[['original address', 'latitude', 'longitude']]
geo_fix.head()

,original address,latitude,longitude
0,"6354 W HUTINGTON CHICAGO, IL",41.992800,-87.786801
1,"5358 S ELLSWORTH DR CHICAGO, IL",41.798336,-87.613949
2,"5415 S ELLSWORTH CHICAGO, IL",41.798336,-87.613949
3,"5446 S ELLSWORTH DR CHICAGO, IL",41.798336,-87.613949
4,"5159 S ELLSWORTH DR CHICAGO, IL",41.798336,-87.613949


In [16]:
geo_fix = geo_fix.rename(columns={'original address': 'Violation Address', 'latitude': 'lat', 'longitude': 'lon'})
geo_fix.head()

,Violation Address,lat,lon
0,"6354 W HUTINGTON CHICAGO, IL",41.992800,-87.786801
1,"5358 S ELLSWORTH DR CHICAGO, IL",41.798336,-87.613949
2,"5415 S ELLSWORTH CHICAGO, IL",41.798336,-87.613949
3,"5446 S ELLSWORTH DR CHICAGO, IL",41.798336,-87.613949
4,"5159 S ELLSWORTH DR CHICAGO, IL",41.798336,-87.613949


In [17]:
geocoded_taxi[geocoded_taxi['Violation Address'] == '6354 W HUTINGTON CHICAGO, IL']

,Ticket Number,License Plate Number,License Plate State,License Plate Type,Ticket Make,Issue Date,Violation Location,Violation Code,Violation Description,Badge,Unit,Ticket Queue,Hearing Dispo,Violation Address,lat,lon
4880,59392077,1916TX,IL,TXI,FORD,02/27/2011 10:00 pm,6354 W HUTINGTON,0964170A,"TRUCK,RV,BUS, OR TAXI RESIDENTIAL STREET",7355,16,Paid,Liable,"6354 W HUTINGTON CHICAGO, IL",38.087223,-88.901443


In [21]:
# geocoded_taxi.set_value(
#     geocoded_taxi[geocoded_taxi['Violation Address'] == '6354 W HUTINGTON CHICAGO, IL'].index, 
#     'lat',
#     41.992800
# )
geocoded_taxi.set_value(
    geocoded_taxi[geocoded_taxi['Violation Address'] == '6354 W HUTINGTON CHICAGO, IL'].index, 
    'lon',
    -87.786801
)
geocoded_taxi[geocoded_taxi['Violation Address'] == '6354 W HUTINGTON CHICAGO, IL']

,Ticket Number,License Plate Number,License Plate State,License Plate Type,Ticket Make,Issue Date,Violation Location,Violation Code,Violation Description,Badge,Unit,Ticket Queue,Hearing Dispo,Violation Address,lat,lon
4880,59392077,1916TX,IL,TXI,FORD,02/27/2011 10:00 pm,6354 W HUTINGTON,0964170A,"TRUCK,RV,BUS, OR TAXI RESIDENTIAL STREET",7355,16,Paid,Liable,"6354 W HUTINGTON CHICAGO, IL",41.9928,-87.786801


In [24]:
geo_dict = fixed_geo.to_dict(orient='records')
print(len(geo_dict))
geo_dict

18


[{'accuracy': 9,
  'latitude': 41.9928,
  'longitude': -87.786801,
  'original address': '6354 W HUTINGTON CHICAGO, IL',
  'returned address': '6354 W Huntington St, Chicago, IL 60646, USA'},
 {'accuracy': 8,
  'latitude': 41.798336,
  'longitude': -87.613949,
  'original address': '5358 S ELLSWORTH DR CHICAGO, IL',
  'returned address': '5358 S Ellsworth Dr, Chicago, IL 60615, USA'},
 {'accuracy': 8,
  'latitude': 41.798336,
  'longitude': -87.613949,
  'original address': '5415 S ELLSWORTH CHICAGO, IL',
  'returned address': '5415 S Ellsworth Dr, Chicago, IL 60615, USA'},
 {'accuracy': 8,
  'latitude': 41.798336,
  'longitude': -87.613949,
  'original address': '5446 S ELLSWORTH DR CHICAGO, IL',
  'returned address': '5446 S Ellsworth Dr, Chicago, IL 60615, USA'},
 {'accuracy': 8,
  'latitude': 41.798336,
  'longitude': -87.613949,
  'original address': '5159 S ELLSWORTH DR CHICAGO, IL',
  'returned address': '5159 S Ellsworth Dr, Chicago, IL 60615, USA'},
 {'accuracy': 8,
  'latitud

In [25]:
geo_dict = geo_dict[1:]
print(len(geo_dict))

17


In [26]:
for g in geo_dict:
    geocoded_taxi.set_value(
        geocoded_taxi[geocoded_taxi['Violation Address'] == g['original address']].index, 
        'lat',
        g['latitude']
    )
    geocoded_taxi.set_value(
        geocoded_taxi[geocoded_taxi['Violation Address'] == g['original address']].index, 
        'lon',
        g['longitude']
    )

In [27]:
geocoded_taxi[geocoded_taxi['lat'] < 41].count()

Ticket Number            0
License Plate Number     0
License Plate State      0
License Plate Type       0
Ticket Make              0
Issue Date               0
Violation Location       0
Violation Code           0
Violation Description    0
Badge                    0
Unit                     0
Ticket Queue             0
Hearing Dispo            0
Violation Address        0
lat                      0
lon                      0
dtype: int64

In [28]:
geocoded_taxi.to_csv('taxi_tickets_geocoded.csv', index=False)

In [29]:
geocoded_taxi['Violation Description'].unique()

array(['EXPIRED METER OR OVERSTAY',
       'TRUCK,MOTOR HOME, BUS BUSINESS STREET',
       'TRUCK,RV,BUS, OR TAXI RESIDENTIAL STREET',
       'EXPIRED PLATES OR TEMPORARY REGISTRATION',
       'PARK OR STAND IN BUS/TAXI/CARRIAGE STAND',
       'PARKING/STANDING PROHIBITED ANYTIME', 'RUSH HOUR PARKING',
       'DOUBLE PARKING OR STANDING', 'NO STANDING/PARKING TIME RESTRICTED',
       'PARK OUTSIDE METERED SPACE', 'OBSTRUCT ROADWAY',
       'RESIDENTIAL PERMIT PARKING', "WITHIN 15' OF FIRE HYDRANT",
       'STREET CLEANING', 'REAR AND FRONT PLATE REQUIRED',
       'STAND, PARK, OR OTHER USE OF BUS LANE',
       'PARK OR STAND ON CROSSWALK', 'CURB LOADING ZONE',
       'DISABLED CURB CUT', 'SAFETY BELTS REQUIRED',
       'BLOCK ACCESS/ALLEY/DRIVEWAY/FIRELANE', '3-7 AM SNOW ROUTE',
       'PARK OR STAND ON SIDEWALK',
       "TWO HEAD LAMPS REQUIRED VISIBLE 1000'",
       'PARK OR STAND ON CHA PROPERTY', 'NO PARKING IN LOOP',
       'NO CITY STICKER OR IMPROPER DISPLAY',
       "WRONG DIRE

In [30]:
res_taxi_tickets = geocoded_taxi[geocoded_taxi['Violation Description'] == 'TRUCK,RV,BUS, OR TAXI RESIDENTIAL STREET']
print(res_taxi_tickets.shape)
res_taxi_tickets.head()

(39096, 16)


,Ticket Number,License Plate Number,License Plate State,License Plate Type,Ticket Make,Issue Date,Violation Location,Violation Code,Violation Description,Badge,Unit,Ticket Queue,Hearing Dispo,Violation Address,lat,lon
2,55388795,25495TX,IL,TXI,MERC,01/05/2009 04:19 am,5721 W EASTWOOD,0964170A,"TRUCK,RV,BUS, OR TAXI RESIDENTIAL STREET",13250,16,Notice,NaN,"5721 W EASTWOOD CHICAGO, IL",41.964930,-87.770331
15,55984582,3529TX,IL,TXI,MERZ,01/18/2009 10:04 pm,2822 W BRYN MAWR,0964170A,"TRUCK,RV,BUS, OR TAXI RESIDENTIAL STREET",19838,20,Paid,NaN,"2822 W BRYN MAWR CHICAGO, IL",41.983193,-87.699957
22,55772826,1262TX,IL,TXI,FORD,01/19/2009 07:30 am,6320 RIDGE,0964170A,"TRUCK,RV,BUS, OR TAXI RESIDENTIAL STREET",17392,24,Paid,NaN,"6320 RIDGE CHICAGO, IL",42.010156,-87.682322
27,56126178,1515TX,IL,TXI,FORD,02/08/2009 02:53 pm,4219 N MILWAUKEE,0964170A,"TRUCK,RV,BUS, OR TAXI RESIDENTIAL STREET",11128,16,Paid,Liable,"4219 N MILWAUKEE CHICAGO, IL",41.957540,-87.751636
42,56062981,5410TX,IL,TXI,FORD,02/12/2009 06:10 am,6935 THOME,0964170A,"TRUCK,RV,BUS, OR TAXI RESIDENTIAL STREET",17392,24,Dismissed,DOR Withdraw,"6935 THOME CHICAGO, IL",41.994834,-87.739869


In [31]:
res_taxi_tickets.to_csv('residential_taxi_tickets.csv', index=False)

In [ ]:
taxi_categories